In [74]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import glob
import pandas as pd
from numpy import random
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [55]:
transcripts = glob.glob('../data/Biden/*.txt')

In [56]:
trainFrac = int(len(transcripts)*0.9)

training = transcripts[:trainFrac]
testing = transcripts[trainFrac:]

In [57]:
speeches = []
for i in range(len(training)):
    speech = open(training[i],'r')
    words = []
    for line in speech:
        words += word_tokenize(line)
    seperator = ' '
    speeches.append(seperator.join(words))

In [69]:
cv = CountVectorizer(analyzer = 'word', max_features = 50, lowercase=True, preprocessor=None, tokenizer=None, stop_words = 'english')  
vectors = cv.fit_transform(speeches)
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 0)
kmean_indices = kmeans.fit_predict(vectors)

In [70]:
print(kmean_indices)
transcripts

[2 2 2 0 2 2 0 2 2 2 0 1 2 2 2 0 2 0 2]


['../data/Biden/March122020.txt',
 '../data/Biden/June092020.txt',
 '../data/Biden/May292020.txt',
 '../data/Biden/April072020.txt',
 '../data/Biden/October092019.txt',
 '../data/Biden/July042020.txt',
 '../data/Biden/July142020.txt',
 '../data/Biden/June252020.txt',
 '../data/Biden/June022020.txt',
 '../data/Biden/March102020.txt',
 '../data/Biden/July212020.txt',
 '../data/Biden/May042020.txt',
 '../data/Biden/March232020.txt',
 '../data/Biden/July022020.txt',
 '../data/Biden/June172020.txt',
 '../data/Biden/July032020.txt',
 '../data/Biden/February292020.txt',
 '../data/Biden/July082020.txt',
 '../data/Biden/August082019.txt',
 '../data/Biden/July092020.txt',
 '../data/Biden/February172020.txt',
 '../data/Biden/March172020.txt']

In [71]:
pca = PCA(n_components=2)
scatter_plot_points = pca.fit_transform(vectors.toarray())

In [72]:
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()
for i in range(3):
    print("Cluster %d: " % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),

Cluster 0: 
 going
 president
 people
 know
 workers
 make
 thank
 need
 country
 work
Cluster 1: 
 workers
 president
 thank
 people
 going
 work
 like
 make
 know
 need
Cluster 2: 
 president
 trump
 people
 need
 type
 nation
 american
 america
 url
 going


In [63]:
#colors = ["r", "b", "c" ,"m"]

#x_axis = [o[0] for o in scatter_plot_points]
#y_axis = [o[1] for o in scatter_plot_points]
#fig, ax = plt.subplots(figsize=(20,10))
#ax.scatter(x_axis, y_axis, c=[colors[d] for d in kmean_indices])

#for i, txt in enumerate(speeches):
#    ax.annotate(txt, (x_axis[i], y_axis[i]))